In [14]:
import pandas as pd
import requests
import json
import time
from git import Repo
import re
from pathlib import Path
import git

In [15]:
tr_data = pd.read_csv('data/extracted_project_travis/gradle.csv')
tr_data = tr_data[ tr_data['git_branch'] == 'master']
clone_dir = 'project_clones/gradle'

In [16]:
fe = open('file_extensions.json', 'r')
file_extensions = json.load(fe)
types = list(file_extensions.keys())

In [17]:
repo = Repo(clone_dir)

In [18]:
def ignoreTheFormate(string_code):
    int1 = re.sub('/\/\/(.*)|\/\*(\*(?!\/)|[^*])*\*\//g', '', string_code) #removing single and multiline java comments
    int2 = re.sub('/([\\\]+<>=*(){}[,;-])/g', "\n$1\n", int1) 
    int3 = re.sub('/\s(\s*)/g', "\n", int2) #removing white spaces
    return int3.strip()


In [19]:
def getType(extension):
    
    for i in range(len(types)):
        if extension in file_extensions[types[i]]:
            return types[i]

    if (extension == ''):
        file_name = path.basename(file)
        if (file_name.indexOf('.git') == 0 |
            file_name.indexOf('.bzr') == 0 |
            file_name.indexOf('.svn') == 0 |
            file_name.indexOf('.cvs') == 0):
            return 'meta'
    
    return "unknown"


In [20]:
rule1 = [] #non_source_code
rule2 = [] #version change (preparing for release)
rule3 = [] #comment change
rule4 = [] #meta file changes
rule5 = [] #formatting
verdict_list = [] #final decision

In [ ]:
for x in range(len(tr_data)):
    row = tr_data.iloc[x]
    commit_id = row['git_trigger_commit']
    print(commit_id)
    
    
    commit = repo.commit(commit_id)
    
    file_flag_list = []
    code_flag_list = []
    format_flag_list = []
    
    verdict = 1
    
#     file_flag = 0    #we begin by assuming all files are code files
#     code_flag = 1    #we begin by assuming all changes are non-code changes
#     format_flag = 0    #we begin by assuming all changes are not format-related

    
    for file_name in commit.stats.files:
        
        extension = '.' + file_name.split('.')[-1]
        t = getType(extension)
        
        if (t == 'meta') | (t == 'media') | (t == 'doc'):
            file_flag_list.append(1)
            verdict = verdict & 1
            #we have found non-source code changes
            #we have found meta file changes
            
            #we have found that this file can be skipped 
        
        elif extension == '.java':
            try:
                new_version = ignoreTheFormate(repo.git.show('{}:{}'.format(commit_id, file_name)))
                old_version = ignoreTheFormate(repo.git.show('{}:{}'.format(commit_id + '~', file_name)))
            
                if old_version != new_version:
                    #we have found code changes
                    #we have found that this file cannot be skipped
                    code_flag_list.append(0)
                    verdict = verdict & 0
                else:
                    #we have found file formatting or comment changes
                    #we have found that this file can be skipped 
                    format_flag_list.append(1)
                    verdict = verdict & 1
            except:
                #file added or deleted
                print('File {} added or deleted'.format(file_name))
                verdict = verdict & 0
            
        else:
            #out of skip rules scope
            verdict = verdict & 0
            continue
    
#     print(file_flag_list)
#     print(format_flag_list)
#     print(code_flag_list)
    verdict_list.append(verdict)
    

cadf37691927f6e754ad0ba6a1811109e24a269d
bb405114610ccc2d3fbeb37b69c245c0d0708c53
eb955b983803f99c898c8071b1165532d6793508
c7d0d40b988c13c05281d33dd35c377db144c71d
99489a6abc9fafb4a690833c0d07f54e2bfd7c17
de25f7a86697a171d980745ab8fd45c10a4ebf3a
612e77473f498e587b40f917fbd0d3d94321be83
53f511086dbb079bc5952c4516ffa92cd3c8cc24
File subprojects/tooling-api/src/main/java/org/gradle/tooling/internal/gradle/TaskListingTaskSelector.java added or deleted
4d7f50cb5aff2e1fbc8296ee4bc7693b8a6e8178
00129787f4ae1313aa31d13026fcc5e154ca9ac4
File subprojects/core-impl/src/main/groovy/org/gradle/api/internal/artifacts/ivyservice/ivyresolve/parser/data/PomProfile.java added or deleted
ca5e403b94b3a36d17613996ab4716325c9564bb
417ddb5cc8e26475b9b5518e21ae728bdbfb577f
55709c83f358867316f1fd729d627082b356ca1e
15e9027310544e05bf5dad17a875799a90775d3c
0ae69a9b60a08d1369219e66337c0294a04c25d2
385ef7abf03406b0faa51327202a0d3621b42bd1
9374690dda4052064c0319a91795cc90ac590e33
File subprojects/plugins/src/main/g

File subprojects/cpp/src/main/groovy/org/gradle/nativebinaries/toolchain/TargetPlatformConfiguration.java added or deleted
File subprojects/cpp/src/main/groovy/org/gradle/nativebinaries/toolchain/TargetPlatformConfigurationInternal.java added or deleted
File subprojects/cpp/src/main/groovy/org/gradle/nativebinaries/toolchain/internal/gcc/ApplyablePlatformConfiguration.java added or deleted
File subprojects/cpp/src/main/groovy/org/gradle/nativebinaries/toolchain/internal/gcc/DefaultConfigurableToolChain.java added or deleted
File subprojects/cpp/src/main/groovy/org/gradle/nativebinaries/toolchain/internal/gcc/TargetPlatformConfiguration.java added or deleted
File subprojects/cpp/src/main/groovy/org/gradle/nativebinaries/toolchain/internal/tools/ConfiguredToolRegistry.java added or deleted
File subprojects/cpp/src/main/groovy/org/gradle/nativebinaries/toolchain/internal/tools/PlatformToolRegistry.java added or deleted
77e3cd028593742335f6edf4f008883efe53a69d
15142dc0a35fd0aeec288f852e8a9

d303fbd0261bb5f7828116f7cd0117529b34c76e
File subprojects/ide/src/main/groovy/org/gradle/plugins/ide/idea/model/internal/IdeaDependenciesProvider.java added or deleted
File subprojects/ide/src/main/groovy/org/gradle/plugins/ide/idea/model/internal/IdeaScopeMappingRule.java added or deleted
File subprojects/ide/src/main/groovy/org/gradle/plugins/ide/idea/model/internal/ModuleDependencyBuilder.java added or deleted
79fae499cd68be32837b639a07edf891f6524962
c09255cc4237fc2aa96354456750f0529fe98a04
7568c5d2f8f6f3977f85c839214fa4115f342d8e
1485dcf7c8a99e3447eb5fb0f2264ff65d5a6a1d
2bb4f8084e9c71616107c38795af6b87f64a9568
File subprojects/cpp/src/main/groovy/org/gradle/nativebinaries/toolchain/internal/AbstractToolChain.java added or deleted
File subprojects/cpp/src/main/groovy/org/gradle/nativebinaries/toolchain/internal/ExtendableToolChain.java added or deleted
File subprojects/cpp/src/main/groovy/org/gradle/nativebinaries/toolchain/internal/tools/ConfigurableToolChainInternal.java added or 

File subprojects/core-impl/src/main/groovy/org/gradle/api/internal/artifacts/result/jvm/DefaultJvmLibrary.java added or deleted
File subprojects/core-impl/src/main/groovy/org/gradle/api/internal/artifacts/result/jvm/DefaultJvmLibraryJavadocArtifact.java added or deleted
File subprojects/core-impl/src/main/groovy/org/gradle/api/internal/artifacts/result/jvm/DefaultJvmLibrarySourcesArtifact.java added or deleted
File subprojects/core/src/main/groovy/org/gradle/api/artifacts/dsl/ArtifactResolutionQuery.java added or deleted
File subprojects/core/src/main/groovy/org/gradle/api/artifacts/resolution/ArtifactResolutionQuery.java added or deleted
File subprojects/core/src/main/groovy/org/gradle/api/artifacts/resolution/ArtifactResolutionQueryResult.java added or deleted
File subprojects/core/src/main/groovy/org/gradle/api/artifacts/resolution/JvmLibrary.java added or deleted
File subprojects/core/src/main/groovy/org/gradle/api/artifacts/resolution/JvmLibraryArtifact.java added or deleted
File 

31701942e876dd19bced7fd9e94852fe80d993b3
db386d2f47fa5dfe2596a68db799d0dfbc39d836
27d96ba8b057ea34835d7446c3b90bd789ce9698
e0cebf61f4ec981efd1c1c01123cf7271bdb7e3a
4fc99598c28e6cbd1ded0a1da80af15a5d7b1d1a
07e77bcc1f7b53ccde000e340227d50fa34d74f3
2517921fbc3fa94ace442ffc22cc64702bd3521e
File subprojects/core/src/main/groovy/org/gradle/initialization/BuildCompletionListener.java added or deleted
dbb117f9fb0843267138a68106b8a40e5fdba355
4210a2ce3f8c4abd504e288eccc9e6e894fe78a1
5eb9703073ddf9f73346caf46929fc5e991b634b
c558ca3816e205cab1f43a29915b2f7371886b13
File subprojects/cpp/src/main/groovy/org/gradle/nativebinaries/toolchain/internal/gcc/PostTransformActionArgsTransformer.java added or deleted
8f7f25fe3b005210c26222512238540d8886eb17
8cc89f6b9b7d66dbc904a93977688294f13ae185
b2c1d2af87300bd78839976f09bd6786727989f5
b690b255ecefe8724fd78e8248d91be054e8e718
32570803a68359a171aafad741e526783793f83e
fe64d99c5d89741efa6b3be6c719c01401226844
a102875beadc4f38d959f7918e84dde97d4cd257
2f7a0b3f9

...

File subprojects/core-impl/src/main/groovy/org/gradle/api/internal/externalresource/metadata/DefaultExternalResourceMetaData.java added or deleted
File subprojects/core-impl/src/main/groovy/org/gradle/api/internal/externalresource/metadata/ExternalResourceMetaData.java added or deleted
File subprojects/core-impl/src/main/groovy/org/gradle/api/internal/externalresource/metadata/ExternalResourceMetaDataCompare.java added or deleted
File subprojects/core-impl/src/main/groovy/org/gradle/api/internal/externalresource/transfer/AbstractProgressLoggingHandler.java added or deleted
File subprojects/core-impl/src/main/groovy/org/gradle/api/internal/externalresource/transfer/CacheAwareExternalResourceAccessor.java added or deleted
File subprojects/core-impl/src/main/groovy/org/gradle/api/internal/externalresource/transfer/DefaultCacheAwareExternalResourceAccessor.java added or deleted
File subprojects/core-impl/src/main/groovy/org/gradle/api/internal/externalresource/transfer/ExternalResourceAcce

File subprojects/core-impl/src/main/groovy/org/gradle/internal/resource/transport/sftp/SftpResourceLister.java added or deleted
File subprojects/core-impl/src/main/groovy/org/gradle/internal/resource/transport/sftp/SftpResourceUploader.java added or deleted
File subprojects/core-impl/src/main/groovy/org/gradle/internal/resource/transport/sftp/SftpTransport.java added or deleted
File subprojects/core-impl/src/test/groovy/org/gradle/api/internal/artifacts/ivyservice/moduleconverter/dependencies/ClientModuleIvyDependencyDescriptorFactoryTest.java added or deleted
File subprojects/core-impl/src/test/groovy/org/gradle/api/internal/artifacts/ivyservice/moduleconverter/dependencies/DefaultClientModuleMetaDataFactoryTest.java added or deleted
File subprojects/core-impl/src/test/groovy/org/gradle/api/internal/artifacts/ivyservice/moduleconverter/dependencies/DefaultDependenciesToModuleDescriptorConverterTest.java added or deleted
File subprojects/core/src/integTest/resources/org/gradle/api/task

File subprojects/core/src/main/groovy/org/gradle/plugin/PluginHandler.java added or deleted
File subprojects/core/src/main/groovy/org/gradle/plugin/internal/DefaultPluginHandler.java added or deleted
File subprojects/core/src/main/groovy/org/gradle/plugin/internal/InvalidPluginIdException.java added or deleted
File subprojects/core/src/main/groovy/org/gradle/plugin/internal/NonPluggableTargetPluginHandler.java added or deleted
File subprojects/core/src/main/groovy/org/gradle/plugin/internal/PluginId.java added or deleted
File subprojects/core/src/main/groovy/org/gradle/plugin/internal/PluginRequestApplicator.java added or deleted
File subprojects/core/src/main/groovy/org/gradle/plugin/internal/PluginResolutionApplicator.java added or deleted
File subprojects/core/src/main/groovy/org/gradle/plugin/internal/PluginResolverFactory.java added or deleted
File subprojects/core/src/main/groovy/org/gradle/plugin/internal/PluginResolvers.java added or deleted
File subprojects/core/src/main/groov

File subprojects/cpp/src/main/groovy/org/gradle/nativebinaries/test/NativeTestSuite.java added or deleted
File subprojects/cpp/src/main/groovy/org/gradle/nativebinaries/test/NativeTestSuiteBinary.java added or deleted
File subprojects/cpp/src/main/groovy/org/gradle/nativebinaries/test/ProjectComponentNativeTestSuite.java added or deleted
File subprojects/cpp/src/main/groovy/org/gradle/nativebinaries/test/ProjectComponentTestSuite.java added or deleted
File subprojects/cpp/src/main/groovy/org/gradle/nativebinaries/test/TestSuite.java added or deleted
File subprojects/cpp/src/main/groovy/org/gradle/nativebinaries/test/internal/DefaultNativeTestSuiteBinary.java added or deleted
File subprojects/cpp/src/main/groovy/org/gradle/nativebinaries/test/internal/DefaultTestSuiteExecutableBinary.java added or deleted
File subprojects/cpp/src/main/groovy/org/gradle/nativebinaries/toolchain/CommandLineToolConfiguration.java added or deleted
File subprojects/cpp/src/main/groovy/org/gradle/nativebinari

File subprojects/language-jvm/src/main/groovy/org/gradle/language/jvm/Classpath.java added or deleted
File subprojects/language-jvm/src/main/groovy/org/gradle/language/jvm/internal/ClassDirectoryBinaryNamingScheme.java added or deleted
File subprojects/language-jvm/src/main/groovy/org/gradle/language/jvm/internal/DefaultClassDirectoryBinary.java added or deleted
File subprojects/language-jvm/src/main/groovy/org/gradle/language/jvm/internal/DefaultClasspath.java added or deleted
File subprojects/language-jvm/src/main/groovy/org/gradle/language/jvm/plugins/JvmLanguagePlugin.java added or deleted
File subprojects/language-jvm/src/main/groovy/org/gradle/language/jvm/plugins/package-info.java added or deleted
File subprojects/language-jvm/src/main/groovy/org/gradle/runtime/jvm/Classpath.java added or deleted
File subprojects/language-jvm/src/main/groovy/org/gradle/runtime/jvm/JvmComponentExtension.java added or deleted
File subprojects/language-jvm/src/main/groovy/org/gradle/runtime/jvm/Jvm

In [ ]:
tr_data['verdict'] = verdict_list

In [ ]:
tr_data.to_csv('skip_verdict_gradle.csv')

In [ ]:
print(verdict_list.count(1), len(verdict_list), 100*verdict_list.count(1)/len(verdict_list))